# UK Dive Data

Run in Terminal

In [74]:
#Run: 
#     easy_install pip 
#     pip install BeautifulSoup4

In [2]:
# import required libraries

import requests
import string
from bs4 import BeautifulSoup
import pandas as pd

Scrape and store shipwreck names and locations

In [3]:
# get alphabet as a list
alphabet = string.ascii_uppercase[:26]

# for each letter in the alphabet, get all wreck names and coordinates
wreck_locations = []
location_conversion = []
base_url = 'http://www.ukdiving.co.uk/wrecks/'
for letter in alphabet:
    pageA = requests.get('http://www.ukdiving.co.uk/wrecks/wreck_list.php?letter=' + letter)
    soup = BeautifulSoup(pageA.text, 'html.parser')
    wreck_elements = soup.findAll("a", {"class": "forum"})
    
    for element in wreck_elements:
        string_element = str(element)
        link = string_element.split('href="')[1].split('">')[0]
        wreck_name = element.text
        wreck_url = base_url + link

        # coordinates
        page = requests.get(wreck_url)
        soup = BeautifulSoup(page.text, 'html.parser')    
        wreck_location = soup.findAll("td", {"class": "diving"})
        location = str(wreck_location[5]).split('</strong>')[1].split('</td>')[0]

        # if wreck doesn't have a location, print unknown or don't include?
        if location != "0 ° 0 ' 0 '' - - 0 ° 0' 0'' -":
            wreck_locations.append([wreck_name]) 
            location_conversion.append(location)

Convert coordinate data to decimal format

In [16]:
actual_conversion = []

for coordinates in location_conversion:
    string_coordinate = str(coordinates)
    if '- -' in string_coordinate:
        latitude = string_coordinate.split('- -')[0]
        longitude = string_coordinate.split('- -')[1]
    else:
        latitude = string_coordinate.split('North - ')[0]
        longitude = string_coordinate.split('- ')[1]

    degrees = int(latitude.split('°')[0])
    minutes = int(latitude.split('°')[1].split("'")[0])
    seconds = int(latitude.split('°')[1].split("'")[1].split("' '")[0])
    
    decimal_lat = float(degrees + (minutes / 60) + (seconds / 3600))
    
    if 'West' in longitude:
        degrees = int(longitude.split('°')[0])
        minutes = int(longitude.split('°')[1].split("'")[0])
        seconds = int(longitude.split('°')[1].split("'")[1].split("' '")[0])
        decimal_long = float(-1 * (degrees + (minutes / 60) + (seconds / 3600)))
    elif 'East' in longitude:
        degrees = int(longitude.split('°')[0])
        minutes = int(longitude.split('°')[1].split("'")[0])
        seconds = int(longitude.split('°')[1].split("'")[1].split("' '")[0])
        decimal_long = float((degrees + (minutes / 60) + (seconds / 3600)))
    else:
        degrees = int(longitude.split('°')[0])
        minutes = int(longitude.split('°')[1].split("'")[0])
        seconds = int(longitude.split('°')[1].split("'")[1].split("' '")[0])
        decimal_long = float(-1 * (degrees + (minutes / 60) + (seconds / 3600)))
    
    actual_conversion.append([decimal_lat, decimal_long])
    coordinates_df = pd.DataFrame(actual_conversion, columns = ['latitude', 'longitude'])

coordinates_df['wreck_name'] = [location[0] for location in wreck_locations]
coordinates_df.head()

,latitude,longitude,wreck_name
0,51.688333,-5.177778,Adamantios J Pithis (The Greek)
1,51.200000,1.508333,Admiral Gardner
2,50.500000,-2.133333,Aeolean Sky
3,50.273611,-3.913611,Aircraft Graveyard
4,55.945833,-4.901944,Akka


Export dataframe to csv (an excel doc)

In [33]:
coordinates_df.to_csv('uk-wreck-dcm.csv', sep=',', encoding='utf-8', index=False)